## C-LSTM

In [34]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os
import gc
import time
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.metrics import roc_auc_score
import gensim.models.keyedvectors as word2vec
from nltk.tokenize import WordPunctTokenizer
from collections import Counter

SEED = 41
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

In [35]:
RAW_DATA_PATH        = '../../dl_nlp/data/jigsaw_toxic/raw/'
PROCESSED_DATA_PATH  = '../../dl_nlp/data/jigsaw_toxic/processed/' 

MAX_LEN = 100

### Load Data

In [36]:
def load_sample():
    return pd.read_csv(os.path.join(PROCESSED_DATA_PATH, 'train_sample.csv'))

def load_full():
    train       = pd.read_csv(os.path.join(RAW_DATA_PATH, 'train.csv'))
    test        = pd.read_csv(os.path.join(RAW_DATA_PATH, 'test.csv'))
    test_labels = pd.read_csv(os.path.join(RAW_DATA_PATH, 'test_labels.csv'))
    
    return train, test, test_labels

In [37]:
%%time
train, _, _ = load_full()

CPU times: user 1.49 s, sys: 152 ms, total: 1.64 s
Wall time: 2.34 s


In [4]:
# %%time
# train = load_sample()

CPU times: user 144 ms, sys: 16 ms, total: 160 ms
Wall time: 422 ms


#### Define target columns

In [38]:
TARGET_COLS = ['toxic', 
               'severe_toxic', 
               'obscene', 
               'threat', 
               'insult', 
               'identity_hate'
              ]

#### Tokenization

In [39]:
# define tokenizer
tokenizer = WordPunctTokenizer()

In [40]:
%%time
train_tokenized_comments = list(map(tokenizer.tokenize, train.comment_text))

CPU times: user 4.28 s, sys: 332 ms, total: 4.61 s
Wall time: 4.6 s


In [41]:
%%time
train.loc[:, 'tokenized_comments'] = list(map(' '.join, map(tokenizer.tokenize, train.comment_text)))

CPU times: user 5.59 s, sys: 3.3 s, total: 8.89 s
Wall time: 8.88 s


### Create vocabulary

#### Create word freq mapping

In [42]:
token_counts = Counter()

for tok_comments in train_tokenized_comments:
    token_counts.update(tok_comments)

In [43]:
# we can put a threshold on the token frequency to reduce the vocabulary
tokens    = {}
min_count = 10

for token, freq in token_counts.items():
    if freq >= min_count:
        tokens[token] = freq
        
print('Size of the vocabulary: {}'.format(len(tokens)))

Size of the vocabulary: 32838


### Load Pretrained Embeddings

In [44]:
def load_wv_embedding_matrix(words):
    word2vec_dict = word2vec.KeyedVectors.load_word2vec_format('../../dl_nlp/data/jigsaw_toxic/processed/word2vec.bin.gz', binary=True)
    embed_size    = 300

    embedding_index = dict()
    for word in word2vec_dict.wv.vocab:
        embedding_index[word] = word2vec_dict.word_vec(word)

    print('Loaded %d word vectors'%(len(embedding_index)))

    all_embs          = np.stack(list(embedding_index.values()))
    emb_mean, emb_std = all_embs.mean(), all_embs.std()

    UNK, PAD       = 'UNK', 'PAD'
    UNK_IX, PAD_IX = len(words), len(words) + 1

    nb_words = len(words) + 2

    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))

    embed_cnt = 0
    for i, word in enumerate(list(words.keys()) + [UNK, PAD]):
        embedding_vector = embedding_index.get(word)

        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            embed_cnt +=1

    print('total embedded ', embed_cnt, ' common words')
    del embedding_index
    gc.collect()

    return embedding_matrix, UNK, PAD, UNK_IX, PAD_IX

### Token to ID mapping

In [45]:
# token to index (manual)
# UNK, PAD       = 'UNK', 'PAD'
# UNK_IX, PAD_IX =  0, 1

# token_to_id = {UNK: UNK_IX,
#                PAD: PAD_IX
#               }

# for token in tokens.keys():
#     token_to_id[token] = len(token_to_id)

In [46]:
%%time

# token to index ( word2vec embeddings )
embedding_matrix, UNK, PAD, UNK_IX, PAD_IX = load_wv_embedding_matrix(tokens)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


Loaded 3000000 word vectors
total embedded  29714  common words
CPU times: user 2min 2s, sys: 6.79 s, total: 2min 9s
Wall time: 2min 6s


In [47]:
token_to_id      = {word: index for index, word in enumerate(tokens.keys())}
token_to_id[UNK] = UNK_IX
token_to_id[PAD] = PAD_IX

### Pad Sequences and convert map tokens to indices

In [48]:
UNK_IX, PAD_IX = map(token_to_id.get, [UNK, PAD])

def as_matrix(sequences, token_to_id, word_dropout, UNK_IX, PAD_IX, max_len=None):
    """ Convert a list of tokens into a matrix with padding """

    if isinstance(sequences[0], str):
        sequences = list(map(str.split, sequences))

    max_len = min(max(map(len, sequences)), max_len or float('inf'))
    matrix = np.full((len(sequences), max_len), np.int32(PAD_IX))

    for i,seq in enumerate(sequences):
        row_ix = [token_to_id.get(word, UNK_IX) for word in seq[:max_len]]
        matrix[i, :len(row_ix)] = row_ix

    if word_dropout != 0:
        matrix = apply_word_dropout(matrix, 1 - word_dropout, replace_with=UNK_IX, pad_ix=PAD_IX)

    return matrix

def apply_word_dropout(matrix, keep_prop, replace_with, pad_ix):
    dropout_mask = np.random.choice(2, np.shape(matrix), p=[keep_prop, 1-keep_prop])
    dropout_mask &= matrix != pad_ix
    return np.choose(dropout_mask, [matrix, np.full_like(matrix, replace_with)])

### Split data into training and validation split

In [49]:
from sklearn.model_selection import train_test_split

data_train, data_val = train_test_split(train, test_size=0.2, random_state=42)
data_train.index     = range(len(data_train))
data_val.index       = range(len(data_val))

print("Train size = ", len(data_train))
print("Validation size = ", len(data_val))

Train size =  127656
Validation size =  31915


### Data loader

In [50]:
def iterate_batches(matrix, labels, batch_size, predict_mode='train'):
    indices = np.arange(len(matrix))
    if predict_mode == 'train':
        np.random.shuffle(indices)
    
    for start in range(0, len(matrix), batch_size):
        end = min(start + batch_size, len(matrix))
        
        batch_indices = indices[start: end]
        X = matrix[batch_indices]
        
        if predict_mode != 'train': yield X
        else: yield X, labels[batch_indices]

### Model Definition

In [97]:
class Net(nn.Module):
    def __init__(self, weights, vocab_size, embed_dim, hidden_dim, PAD_IX):
        super(Net, self).__init__()
        
        self.embed_dim   = embed_dim
        self.hidden_dim  = hidden_dim
        
        self.nfms        = [150, 150, 150]
        self.ks          = [3, 3, 5]
        self.Cout        = 1
        
        self.embedding        = nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight = nn.Parameter(weights)
        self.embedding.weight.requires_grad = False
        
        self.ns_embedding        = nn.Embedding(vocab_size, embed_dim)
        self.ns_embedding.weight = nn.Parameter(weights)
        
        # convolutional layer
        self.conv_layer1 = nn.Conv1d(self.Cout * self.embed_dim, self.nfms[0], self.ks[0], padding=1)
        self.conv_layer2 = nn.Conv1d(self.Cout * self.embed_dim, self.nfms[1], self.ks[1], padding=1)
        self.conv_layer3 = nn.Conv1d(self.Cout * self.embed_dim, self.nfms[2], self.ks[2], padding=2)
        
        self.relu        = nn.ReLU()
        
        # LSTM cell
        self.lstm      = nn.LSTM(150, hidden_dim)
#         self.fc1       = nn.Linear(hidden_dim, hidden_dim)
#         self.dropout   = nn.Dropout(0.1)
        self.fc2       = nn.Linear(hidden_dim, 6)
        
    def forward(self, x):
        s_emb  = self.embedding(x)
#         s_emb  = torch.transpose(s_emb, 1, 2)
        
#         ns_emb = self.ns_embedding(x) 
#         ns_emb = torch.transpose(ns_emb, 1, 2)
        
#         out    = torch.cat((s_emb, ns_emb), dim=1)
        
#         # transpose from batch, width, channels to batch, channels, width
        out = torch.transpose(s_emb, 1, 2)
        
#         print('shape after creating multi channel ', out.shape)
        
        # pass it throught conv layer
        out1 = self.conv_layer1(out)
        out1 = self.relu(out1)
        
        out2 = self.conv_layer2(out)
        out2 = self.relu(out2)
        
        out3 = self.conv_layer3(out)
        out3 = self.relu(out3)
        
        
        # change it back to original format
        out1 = torch.transpose(out1, 1, 2)
        out2 = torch.transpose(out2, 1, 2)
        out3 = torch.transpose(out3, 1, 2)
        
        out  = torch.cat((out1, out2, out3), dim=1)
        
        # input format should be (sequence, batch, elements)
        out = torch.transpose(out, 0, 1)
        
        # pass it through LSTM cell
        out, (ht, ct) = self.lstm(out)
        
        # max pooling over time
        out = out.max(dim=0)[0]
        
        # pass it to fully connected layer
#         out = self.fc1(out)
#         out = self.dropout(out)
        
        # final dense layer
        out = self.fc2(out)
        
        return out

### Training Loop

In [81]:
def do_epoch(model, criterion, data, batch_size, optimizer=None):
    epoch_loss, total_size = 0, 0
    per_label_preds = [[], [], [], [], [], []]
    per_label_true  = [[], [], [], [], [], []]
    
    is_train = not optimizer is None
    model.train(is_train)
    
    data, labels = data
    batchs_count = math.ceil(data.shape[0] / batch_size)
    
    with torch.autograd.set_grad_enabled(is_train):
        for i, (X_batch, y_batch) in enumerate(iterate_batches(data, labels, batch_size)):
            X_batch, y_batch = torch.cuda.LongTensor(X_batch), torch.cuda.FloatTensor(y_batch)

            logits = model(X_batch)
            loss   = criterion(logits, y_batch)
            
            if is_train:
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
            
            # convert true target
            batch_target = y_batch.cpu().detach().numpy()
            logits_cpu   = logits.cpu().detach().numpy()
            
            # per_label_preds
            for j in range(6):
                label_preds     = logits_cpu[:, j]
                per_label_preds[j].extend(label_preds)
                per_label_true[j].extend(batch_target[:, j])
                            
            # calculate log loss
            epoch_loss += loss.item()

            print('\r[{} / {}]: Loss = {:.4f}'.format(
                  i, batchs_count, loss.item(), end=''))
    
    label_auc = []
    
    for i in range(6):
        label_auc.append(roc_auc_score(per_label_true[i], per_label_preds[i]))
    
    return epoch_loss / batchs_count, np.mean(label_auc)

def fit(model, criterion, optimizer, train_data, epochs_count=1, 
        batch_size=32, val_data=None, val_batch_size=None):
    if not val_data is None and val_batch_size is None:
        val_batch_size = batch_size
        
    for epoch in range(epochs_count):
        start_time = time.time()
        train_loss, train_auc = do_epoch(
            model, criterion, train_data, batch_size, optimizer
        )
        
        output_info = '\rEpoch {} / {}, Epoch Time = {:.2f}s: Train Loss = {:.4f}, Train AUC = {:.4f}'
        if not val_data is None:
            val_loss, val_auc   = do_epoch(model, criterion, val_data, val_batch_size, None)
            
            epoch_time   = time.time() - start_time
            output_info += ', Val Loss = {:.4f}, Val AUC = {:.4f}'
            print(output_info.format(epoch+1, epochs_count, epoch_time, 
                                     train_loss,
                                     train_auc,
                                     val_loss,
                                     val_auc
                                    ))
        else:
            epoch_time = time.time() - start_time
            print(output_info.format(epoch+1, epochs_count, epoch_time, train_loss, train_auc))

### Run on a single batch

In [56]:
matrix = as_matrix(data_train['tokenized_comments'], 
                   token_to_id, 
                   word_dropout=0, 
                   UNK_IX=UNK_IX, 
                   PAD_IX=PAD_IX,
                   max_len=MAX_LEN
                  )

labels = data_train.loc[:, TARGET_COLS].values
X, y   = next(iterate_batches(matrix, labels, batch_size=2))

In [57]:
X = torch.cuda.LongTensor(X)
y = torch.cuda.LongTensor(y)

In [98]:
vocab_size = len(token_to_id)
embed_dim  = 300
hidden_dim = 16

model = Net(torch.FloatTensor(embedding_matrix), 
             vocab_size, 
             embed_dim,
             hidden_dim,
             PAD_IX).cuda()

In [99]:
logits = model(X)
print(logits)

tensor([[-0.1728, -0.1868,  0.0962,  0.1656, -0.1425, -0.1969],
        [-0.1577, -0.1923,  0.1070,  0.1652, -0.1362, -0.1874]],
       device='cuda:0', grad_fn=<AddmmBackward>)


### Run on full batch

In [100]:
vocab_size = len(token_to_id)
embed_dim  = 300
hidden_dim = 64

model        = Net(torch.FloatTensor(embedding_matrix), 
                    vocab_size,
                    embed_dim,
                    hidden_dim, 
                    PAD_IX
                    ).cuda()

criterion    = nn.BCEWithLogitsLoss().cuda()
optimizer    = optim.Adam([param for param in model.parameters() if param.requires_grad], lr=0.001)

X_train      = as_matrix(data_train['tokenized_comments'], 
                         token_to_id, 
                         word_dropout=0.001, 
                         UNK_IX=UNK_IX, 
                         PAD_IX=PAD_IX,
                         max_len=MAX_LEN
                        )

train_labels = data_train.loc[:, TARGET_COLS].values 

X_test       = as_matrix(data_val['tokenized_comments'],
                         token_to_id, 
                         word_dropout=0.001, 
                         UNK_IX=UNK_IX, 
                         PAD_IX=PAD_IX,
                         max_len=MAX_LEN
                        )

test_labels  = data_val.loc[:, TARGET_COLS].values

fit(model, criterion, optimizer, train_data=(X_train, train_labels), epochs_count=10, 
    batch_size=512, val_data=(X_test, test_labels), val_batch_size=1024)

[0 / 250]: Loss = 0.7284
[1 / 250]: Loss = 0.7124
[2 / 250]: Loss = 0.6942
[3 / 250]: Loss = 0.6752
[4 / 250]: Loss = 0.6570
[5 / 250]: Loss = 0.6352
[6 / 250]: Loss = 0.6189
[7 / 250]: Loss = 0.6001
[8 / 250]: Loss = 0.5811
[9 / 250]: Loss = 0.5632
[10 / 250]: Loss = 0.5350
[11 / 250]: Loss = 0.5206
[12 / 250]: Loss = 0.4893
[13 / 250]: Loss = 0.4760
[14 / 250]: Loss = 0.4542
[15 / 250]: Loss = 0.4216
[16 / 250]: Loss = 0.4136
[17 / 250]: Loss = 0.3848
[18 / 250]: Loss = 0.3608
[19 / 250]: Loss = 0.3424
[20 / 250]: Loss = 0.3309
[21 / 250]: Loss = 0.3178
[22 / 250]: Loss = 0.3101
[23 / 250]: Loss = 0.2906
[24 / 250]: Loss = 0.2735
[25 / 250]: Loss = 0.2604
[26 / 250]: Loss = 0.2591
[27 / 250]: Loss = 0.2372
[28 / 250]: Loss = 0.2506
[29 / 250]: Loss = 0.2267
[30 / 250]: Loss = 0.2215
[31 / 250]: Loss = 0.2319
[32 / 250]: Loss = 0.1866
[33 / 250]: Loss = 0.2054
[34 / 250]: Loss = 0.2053
[35 / 250]: Loss = 0.1773
[36 / 250]: Loss = 0.1744
[37 / 250]: Loss = 0.1722
[38 / 250]: Loss = 0.2